# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
Tools allow agents to 'Act' in the real world. 
Careful descriptions can help your agent discover how to use your tools.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/javascript/langchain/tools) for more information.

## Calculator example

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

In [ ]:
import { z } from "zod";
import { tool } from "langchain";

const realNumberCalculator = tool(({ a, b, operation }) => {
    console.log("🧮 Invoking calculator tool")
    if (operation === "add") {
        return a + b;
    } else if (operation === "subtract") {
        return a - b;
    } else if (operation === "multiply") {
        return a * b;
    } else if (operation === "divide") {
        if (b === 0) {
            throw new Error("Division by zero is not allowed.");
        }
        return a / b;
    } else {
        throw new Error(`Invalid operation: ${operation}`);
    }
}, {
    name: "real_number_calculator",
    description: "Perform basic arithmetic operations on two real numbers.",
    schema: z.object({
        a: z.number(),
        b: z.number(),
        operation: z.enum(["add", "subtract", "multiply", "divide"])
    })
});

Now we can create our agent with both tools and the system prompt.


In [ ]:
import * as setup from "./setup.ts";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5-mini",
    tools: [realNumberCalculator],
    systemPrompt: "You are a helpful assistant."
});

This invokes your calculator tool.

In [ ]:
const result = await agent.invoke({
    messages: "What is 3.1125 * 4.1234?"
});

const { md } = Deno.jupyter
md`${result.messages.at(-1).content}`

We can check the **metadata** in [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) to see this.

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

In [ ]:
const result = await agent.invoke({
    messages: "What is 3 * 4?"
});

const { md } = Deno.jupyter
md`${result.messages.at(-1).content}`

This often does not invoke the tool though the input are real numbers. (results vary from run to run)

In [ ]:
const result = await agent.invoke({
    messages: "What is 3.0 * 4.0?"
});

const { md } = Deno.jupyter
md`${result.messages.at(-1).content}`

Change the construction of the tool to force the tool to be called